In [1]:
import os
import sys
import csv
import numpy as np
from matplotlib import pyplot as plt
import math

In [2]:
key_dir = "data/skelton/keys/"
video_name = "hello_a"
key_path = key_dir+video_name+".csv"

In [3]:
img_id,p_id,xpos,ypos,pos_name=[],[],[],[],[]
with open(key_path,"r") as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        if len(line)>0:
            img_id.append(line[0])
            p_id.append(line[1])
            xpos.append(line[2])
            ypos.append(line[3])
            pos_name.append(line[4])
    csvfile.close()

In [4]:
for i in range(len(xpos)):
    ypos[i]=ypos[i][1:-1].split(',')
    xpos[i]=xpos[i][1:-1].split(',')
    pos_name[i] = pos_name[i][1:-1].split(',')
    for j in range(len(xpos[i])):
        xpos[i][j] = float(xpos[i][j].strip())
        ypos[i][j] = -float(ypos[i][j].strip())
        pos_name[i][j] = pos_name[i][j].strip().strip("'")

In [12]:
all_keys=[]
for i in range(len(img_id)):
    keys,keys_dict= {},{}
    for j in range(len(pos_name[i])):
        key_dict={pos_name[i][j]:[xpos[i][j],ypos[i][j]]}
        keys.update(key_dict)
    keys_dict = {img_id[i]+'_'+p_id[i]:keys}
    all_keys.append(keys_dict)
all_keys

[{'6_0': {'nose': [410.0, -580.0],
   'neck': [420.0, -660.0],
   'r_sho': [350.0, -660.0],
   'r_elb': [320.0, -780.0],
   'r_wri': [320.0, -870.0],
   'l_sho': [490.0, -670.0],
   'l_elb': [520.0, -780.0],
   'l_wri': [530.0, -870.0],
   'r_hip': [370.0, -890.0],
   'r_knee': [370.0, -1060.0],
   'r_ank': [370.0, -1220.0],
   'l_hip': [460.0, -890.0],
   'l_knee': [450.0, -1050.0],
   'l_ank': [450.0, -1220.0],
   'r_eye': [400.0, -570.0],
   'l_eye': [420.0, -570.0],
   'r_ear': [380.0, -570.0],
   'l_ear': [450.0, -580.0]}},
 {'12_0': {'nose': [410.0, -580.0],
   'neck': [420.0, -660.0],
   'r_sho': [340.0, -660.0],
   'r_elb': [300.0, -770.0],
   'r_wri': [240.0, -840.0],
   'l_sho': [490.0, -670.0],
   'l_elb': [520.0, -780.0],
   'l_wri': [520.0, -870.0],
   'r_hip': [370.0, -890.0],
   'r_knee': [370.0, -1050.0],
   'r_ank': [370.0, -1220.0],
   'l_hip': [460.0, -890.0],
   'l_knee': [450.0, -1050.0],
   'l_ank': [450.0, -1220.0],
   'r_eye': [400.0, -570.0],
   'l_eye': [420.0

In [6]:
# keys = []
# for i in range(len(img_id)):
#     key=[]
#     key.append(img_id[i])
#     key.append(p_id[i])
#     key.append(xpos[i])
#     key.append(ypos[i])
#     key.append(pos_name[i])
#     keys.append(key)

In [7]:
x=np.array(keys[20][2])
y=np.array(keys[20][3])
plt.figure(figsize=(20,20))
plt.plot(x,y,"o")
ax = plt.gca()
ax.set_aspect(1)
for i in range(len(x)):
    plt.text(x[i], y[i], keys[20][4][i])
plt.savefig("test")

KeyError: 20

In [29]:
joints_dict={'nose2neck2r_sho':[],
                'nose2neck2l_sho':[],
                'neck2r_sho2r_elb':[],
                'neck2l_sho2l_elb':[],
                'r_sho2r_elb2r_wri':[],
                'l_sho2l_elb2l_wri':[],
                'neck2r_hip2r_knee':[],
                'neck2l_hip2l_knee':[]}
list(joints_dict.keys())

['nose2neck2r_sho',
 'nose2neck2l_sho',
 'neck2r_sho2r_elb',
 'neck2l_sho2l_elb',
 'r_sho2r_elb2r_wri',
 'l_sho2l_elb2l_wri',
 'neck2r_hip2r_knee',
 'neck2l_hip2l_knee']

In [30]:
def cal_ang(point_1, point_2, point_3):
    """
    根据三点坐标计算夹角
    :return: 返回任意角的夹角值，这里只是返回点2的夹角
    """
    a=math.sqrt((point_2[0]-point_3[0])*(point_2[0]-point_3[0])+(point_2[1]-point_3[1])*(point_2[1] - point_3[1]))
    b=math.sqrt((point_1[0]-point_3[0])*(point_1[0]-point_3[0])+(point_1[1]-point_3[1])*(point_1[1] - point_3[1]))
    c=math.sqrt((point_1[0]-point_2[0])*(point_1[0]-point_2[0])+(point_1[1]-point_2[1])*(point_1[1]-point_2[1]))
    B=math.degrees(math.acos((b*b-a*a-c*c)/(-2*a*c)))
    return B

joint_name = 'neck2r_sho2r_elb'
"""获取某个关节的时序转动角"""
def get_joint_rotate(joint_name):
    frame_len=len(all_keys) #获取总共帧的个数
    joint_rotate = []
    for frame_id in range(frame_len):
        skelton_id = list(all_keys[frame_id].keys())[0] #默认已经过滤，单帧中骨骼目标为0号
        key_s,key_m,key_e = joint_name.split('2')[0],joint_name.split('2')[1],joint_name.split('2')[2]
        key_s_pos = all_keys[frame_id].get(skelton_id).get(key_s)
        key_m_pos = all_keys[frame_id].get(skelton_id).get(key_m)
        key_e_pos = all_keys[frame_id].get(skelton_id).get(key_e)
        joint_rotate.append(cal_ang(key_s_pos,key_m_pos,key_e_pos))
    return joint_rotate

def get_joints_rotate(joints_dict):
    joints_name = list(joints_dict.keys())
    for joint_name in joints_name:

        joint_rotate = get_joint_rotate(joint_name)
        joints_dict.update({joint_name:joint_rotate})
    return joints_dict

{'nose2neck2r_sho': [82.87498365109822, 82.87498365109822, 82.87498365109822, 82.87498365109822, 82.87498365109822, 90.0, 90.0, 97.12501634890181, 104.03624346792647, 97.12501634890181, 97.12501634890181, 90.0, 90.0, 90.0, 97.12501634890181, 90.0, 97.12501634890181, 97.12501634890181, 97.12501634890181, 97.12501634890181, 90.0, 97.12501634890181, 90.0, 90.0, 90.0, 97.12501634890181, 96.34019174590989, 81.86989764584403, 90.0], 'nose2neck2l_sho': [105.25511870305778, 105.25511870305778, 105.25511870305778, 105.25511870305778, 105.25511870305778, 90.0, 97.1250163489018, 90.00000000000001, 83.08877288097533, 90.00000000000001, 90.00000000000001, 97.1250163489018, 97.1250163489018, 97.1250163489018, 90.00000000000001, 97.1250163489018, 90.00000000000001, 90.00000000000001, 90.00000000000001, 90.00000000000001, 97.1250163489018, 90.00000000000001, 97.1250163489018, 97.1250163489018, 97.1250163489018, 90.00000000000001, 90.78482460299189, 98.13010235415598, 98.13010235415598], 'neck2r_sho2r_

In [32]:
joints_dict.keys()

KeyError: 0